Calculs de la moyenne et de l'écart type par bande sur une série d'images

In [ ]:
from utils.utils import (get_environment,get_root_path,update_storage_access)
import os
import s3fs
from classes.data.satellite_image import SatelliteImage
import numpy as np

In [ ]:
update_storage_access()
root_path = get_root_path()
environment = get_environment()

src = "SENTINEL2"
date = 2022
dep = "976"
nb_bands = 12

In [ ]:
bucket = environment["bucket"]
path_s3 = environment["sources"][src][date][dep]
path_local = os.path.join(
    root_path, environment["local-path"][src][date][dep]
)

fs = s3fs.S3FileSystem(
    client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"}
)
fs.download(
    rpath=f"{bucket}/{path_s3}", lpath=f"{path_local}", recursive=True
)

In [ ]:
path_local = ''
list_paths = os.listdir(path_local)
list_paths = [path_local + '/' + name for name in list_paths]

nb_images = len(list_paths)
mean = [0]*nb_bands
std = [0]*nb_bands

for path in list_paths:
    si = SatelliteImage.from_raster(
        file_path=path,
        dep=dep,
        date=date,
        n_bands=nb_bands
    )

    for i in range(nb_bands):
        if not np.isnan(np.mean(si.array[i])):
            mean[i] += np.mean(si.array[i])
            std[i] += np.std(si.array[i])

for i in range(nb_bands):
    mean[i] /= nb_images
    std[i] /= nb_images

mean=tuple(mean)
std=tuple(std)

print(mean, std)

Calcul et export en .tif du label correspondant à une image d'entrée

In [1]:
from datetime import datetime
import numpy as np
from tqdm import tqdm
import os

from classes.data.satellite_image import SatelliteImage

from train_pipeline_utils.prepare_data import (
    label_images,
    save_images_and_masks,
)
from classes.labelers.labeler import BDTOPOLabeler
from utils.utils import remove_dot_file

In [8]:
year = 2022
dep = "974"
nb_bands=12
task="segmentation"

In [9]:
output_dir = "imgs"
date = datetime.strptime(str(year) + "0101", "%Y%m%d")

dir = '/home/onyxia/work/detection-habitat-spontane/src/donnees_test'
list_path = [dir + "/" + filename for filename in remove_dot_file(os.listdir(dir))]
liste_sorted = np.sort(list_path)
labeler = BDTOPOLabeler(date, dep)

for path in tqdm(liste_sorted):
    si = SatelliteImage.from_raster(
        file_path=path,
        dep=dep,
        date=date,
        n_bands=nb_bands,
    )

    labels, balancing_dict = label_images(
        [si], labeler, task=task
    )

    save_images_and_masks(
        [si],
        labels,
        output_dir,
        'SENTINEL2',
        dir,
        task=task,
    )

download 974 2022 in /home/onyxia/work/detection-habitat-spontane/data/BDTOPO/2022/BDTOPO_REUNION


100%|██████████| 4/4 [00:01<00:00,  2.15it/s]


Création des images test

In [ ]:
import os
from classes.data.satellite_image import SatelliteImage
from osgeo import gdal
import numpy as np

In [ ]:
path_s1 = 'donnees_test/Sentinel1'
path_s2 = 'donnees_test/Sentinel2'
path_s1_2 = 'donnees_test/Sentinel1-2'
path_s1_2_RVB = 'donnees_test/Sentinel1-2-RVB'
path_s2_RVB = 'donnees_test/Sentinel2-RVB'

list_names_s1 = os.listdir(path_s1)
list_names_s2 = os.listdir(path_s2)
list_names_s1_2 = os.listdir(path_s1_2)
list_names_s1_2_RVB = os.listdir(path_s1_2_RVB)
list_names_s2_RVB = os.listdir(path_s2_RVB)

list_paths_s1 = [path_s1+'/'+name for name in list_names_s1]
list_paths_s2 = [path_s2+'/'+name for name in list_names_s2]
list_paths_s1_2 = [path_s1_2+'/'+name for name in list_names_s1_2]
list_paths_s1_2_RVB = [path_s1_2_RVB+'/'+name for name in list_names_s1_2_RVB]
list_paths_s2_RVB = [path_s2_RVB+'/'+name for name in list_names_s2_RVB]

In [ ]:
### Création Sentinel1-2
### Relancer la création des listes de fichiers

for path1 in list_paths_s1:
    full_name = path1.split('/')[-1]
    name1 = path1.split('/')[-1][0:14]
    for path2 in list_paths_s2:
        if name1 in path2:
            image1 = SatelliteImage.from_raster(file_path=path1, dep=None, date=None, n_bands=2)
            image2 = SatelliteImage.from_raster(file_path=path2, dep=None, date=None, n_bands=12)
            array1 = image1.array
            array2 = image2.array

            array1_2 = np.concatenate((array2, array1))
            n_bands = array1_2.shape[0]

            transf = image1.transform
            driver = gdal.GetDriverByName("GTiff")
            out_ds = driver.Create(
                path_s1_2 + "/" + name1 + ".tif",
                array1_2.shape[2],
                array1_2.shape[1],
                array1_2.shape[0],
                gdal.GDT_Float64,
            )
            out_ds.SetGeoTransform(
                [
                    transf[2],
                    transf[0],
                    transf[1],
                    transf[5],
                    transf[3],
                    transf[4],
                ]
            )
            in_ds = gdal.Open(path_s1+'/'+full_name)
            proj = in_ds.GetProjection()
            out_ds.SetProjection(proj)

            for i in range(n_bands):
                out_ds.GetRasterBand(i + 1).WriteArray(array1_2[i, :, :])

            out_ds = None

In [ ]:
### Création Sentinel1-2-RVB
### Relancer la création des listes de fichiers

for path in list_paths_s1_2:
    full_name = path.split('/')[-1]
    name = path.split('/')[-1][0:14]
    image = SatelliteImage.from_raster(file_path=path, dep=None, date=None, n_bands=14)
    array = image.array

    array1_2_RVB = array[(3,2,1,12,13),:,:]
    n_bands = array1_2_RVB.shape[0]

    transf = image.transform
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(
        path_s1_2_RVB + "/" + name + ".tif",
        array1_2_RVB.shape[2],
        array1_2_RVB.shape[1],
        array1_2_RVB.shape[0],
        gdal.GDT_Float64,
    )
    out_ds.SetGeoTransform(
        [
            transf[2],
            transf[0],
            transf[1],
            transf[5],
            transf[3],
            transf[4],
        ]
    )
    in_ds = gdal.Open(path)
    proj = in_ds.GetProjection()
    out_ds.SetProjection(proj)

    for i in range(n_bands):
        out_ds.GetRasterBand(i + 1).WriteArray(array1_2_RVB[i, :, :])

    out_ds = None

In [ ]:
### Création Sentinel2-RVB
### Relancer la création des listes de fichiers

for path in list_paths_s1_2_RVB:
    full_name = path.split('/')[-1]
    name = path.split('/')[-1][0:14]
    image = SatelliteImage.from_raster(file_path=path, dep=None, date=None, n_bands=5)
    array = image.array

    array2_RVB = array[(0, 1, 2),:,:]
    n_bands = array2_RVB.shape[0]

    transf = image.transform
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(
        path_s2_RVB + "/" + name + ".tif",
        array2_RVB.shape[2],
        array2_RVB.shape[1],
        array2_RVB.shape[0],
        gdal.GDT_Float64,
    )
    out_ds.SetGeoTransform(
        [
            transf[2],
            transf[0],
            transf[1],
            transf[5],
            transf[3],
            transf[4],
        ]
    )
    in_ds = gdal.Open(path)
    proj = in_ds.GetProjection()
    out_ds.SetProjection(proj)

    for i in range(n_bands):
        out_ds.GetRasterBand(i + 1).WriteArray(array2_RVB[i, :, :])

    out_ds = None